In [1]:
import visuallayer
visuallayer.__version__

'0.0.13'

In [2]:
from visuallayer.datasets.zoo import VLImageNet1k

In [3]:
imagenet = VLImageNet1k(root='../../imagenet-1k/archives')

In [4]:
imagenet.report

,reason,count,pct
0,Duplicate,7522,0.565
1,dark,3174,0.238
2,blur,2478,0.186
3,Mislabeled,1480,0.111
4,Outlier,1199,0.090
5,leakage,869,0.065
6,bright,770,0.058
7,Total,17492,1.313


In [5]:
imagenet.info

Metadata:
--> Name - vl-imagenet-1k
--> Description - A modified version of the original ImageNet-1k dataset removing dataset issues.
--> License - Unknown
--> Homepage URL - https://www.image-net.org/
--> Number of Images - 1431167
--> Number of Images with Issues - 15997


In [6]:
# imagenet.explore()

In [7]:
train_dataset = imagenet.export(output_format='pytorch', split='train')
len(train_dataset)

Exporting VL dataset into pytorch dataset.
Using provided CSV file: https://sharedvisuallayer.s3.us-east-2.amazonaws.com/visual-layer-sdk/ImageNet-1K_images_issue_file_list.csv


1265871

In [8]:
train_dataset = imagenet.export(output_format='pytorch', split='train', variation='original')
len(train_dataset)

Exporting ORIGINAL dataset into pytorch dataset.


1281167

In [9]:
train_dataset = imagenet.export(output_format='pytorch', split='train')
valid_dataset = imagenet.export(output_format='pytorch', split='val')

Exporting VL dataset into pytorch dataset.
Using provided CSV file: https://sharedvisuallayer.s3.us-east-2.amazonaws.com/visual-layer-sdk/ImageNet-1K_images_issue_file_list.csv
Exporting VL dataset into pytorch dataset.
Using provided CSV file: https://sharedvisuallayer.s3.us-east-2.amazonaws.com/visual-layer-sdk/ImageNet-1K_images_issue_file_list.csv


In [10]:
train_dataset.samples

[('../../imagenet-1k/archives/train/n01440764/n01440764_10026.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10027.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10029.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10040.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10042.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10043.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10048.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10066.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10074.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_1009.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10095.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10108.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440764_10110.JPEG', 0),
 ('../../imagenet-1k/archives/train/n01440764/n01440

In [11]:
len(train_dataset)

1265871

In [12]:
len(valid_dataset)

49299

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision

In [14]:
import torchvision.transforms as transforms

train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

valid_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [15]:
# from vl_datasets import VLImageNet
# train_dataset = VLImageNet('../../imagenet-1k/archives', split='train', exclude_csv='../../imagenet-1k/archives/ImageNet-1K_images_issue_file_list.csv', transform=train_transform)
# valid_dataset = VLImageNet('../../imagenet-1k/archives', split='val', exclude_csv='../../imagenet-1k/archives/ImageNet-1K_images_issue_file_list.csv', transform=valid_transform)

In [16]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=True)

In [17]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

For the purpose of demonstration, we will be training only on 1% of the total images. 

In [19]:
# Define the portion of the dataset you want to train on
subset_ratio = 0.01  # Train on 1% of the dataset
subset_size = int(subset_ratio * len(train_loader.dataset))
subset_sampler = torch.utils.data.SubsetRandomSampler(range(subset_size))
subset_loader = torch.utils.data.DataLoader(train_loader.dataset, sampler=subset_sampler)

In [20]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(subset_loader): # to train on the full dataset replace subset_loader with train_loader
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")

Using device: cpu
Epoch 1 - Loss: 6.116066263683886
Epoch 2 - Loss: 5.885800884129184
Epoch 3 - Loss: 5.374752795774606
Epoch 4 - Loss: 4.1853746764870365
Epoch 5 - Loss: 3.7604093426143557


In [21]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}")

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>